# Subway route planning
In this project, we use graph search algorithm to plan the best subway route. 
- 1. Scrape web data to obtain subway routes, station information and between station distances. 
- 2. Parse, clean and organize station, routes and distances data. 
- 3. The search algorithm to find routes. 
- 4. The comparison algorithm(s) to determine the best route. So far , we can pick paths base on 
    - minimum stops
    - minimum total distances 

## 1. Scrape web data 

We use requests package to get the Beijing Subway website in html form, and use beautifulsoup to parse it to obtain subway routes, station information and between station distances. 

In [2]:
import requests 
subway_main_url="http://www.bjsubway.com/station/xltcx/"
distance_url="http://www.bjsubway.com/station/zjgls/#"
main_page = requests.get(subway_main_url) 
distance_page=requests.get(distance_url)

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(distance_page.content, 'html.parser')

First, try one subway line, whose info are organized under the section with id='sub6'. 

In [53]:
soup.find(id="sub6").select("div  th")

[<th>国家图书馆――白石桥南</th>,
 <th>白石桥南――白堆子</th>,
 <th>白堆子――军事博物馆</th>,
 <th>军事博物馆――北京西站</th>,
 <th>北京西站――六里桥东</th>,
 <th>六里桥东――六里桥</th>,
 <th>六里桥――七里庄</th>,
 <th>七里庄――丰台东大街</th>,
 <th>丰台东大街――丰台南路</th>,
 <th>丰台南路――科怡路</th>,
 <th>科怡路――丰台科技园</th>,
 <th>丰台科技园――郭公庄</th>]

In [71]:
stationlist=[station.get_text() for station in soup.find(id="sub6").select("div th")]
print(stationlist)
print(len(stationlist))

['国家图书馆――白石桥南', '白石桥南――白堆子', '白堆子――军事博物馆', '军事博物馆――北京西站', '北京西站――六里桥东', '六里桥东――六里桥', '六里桥――七里庄', '七里庄――丰台东大街', '丰台东大街――丰台南路', '丰台南路――科怡路', '科怡路――丰台科技园', '丰台科技园――郭公庄']
12


In [70]:
distancelist=[distance.get_text() for distance in soup.find(id="sub6").select("div  td") if distance.get_text().isnumeric() ]
print(distancelist)
print(len(distancelist))

['1096', '943', '1912', '1398', '1170', '1309', '1778', '1325', '1585', '980', '788', '1347']
12


From inspecting the website source we find out that all lines are organized under id=sub0, sub1, ...etc. Make the id list used for looping 

In [4]:
id_list=[]
for i in range(0,18):
    id_list.append('sub'+str(i))
id_list

['sub0',
 'sub1',
 'sub2',
 'sub3',
 'sub4',
 'sub5',
 'sub6',
 'sub7',
 'sub8',
 'sub9',
 'sub10',
 'sub11',
 'sub12',
 'sub13',
 'sub14',
 'sub15',
 'sub16',
 'sub17']

It turns out that there is one page whose format is slightly off.

In [84]:
#this works 
for oneid in id_list:
    stationlist=[station.get_text() for station in soup.find(id=oneid).select("div th")]
    #print(stationlist)
    #print(len(stationlist))
    distancelist=[distance.get_text() for distance in soup.find(id=oneid).select("div  td") if distance.get_text().isnumeric() ]
    #print(distancelist)
    if len(stationlist)!=len(distancelist):
        print('oops we have a problem')
        print(oneid)

oops we have a problem
sub2


In [79]:
oneid='sub2'
stationlist=[station.get_text() for station in soup.find(id=oneid).select("div th")]
stationlist

['起始/终到车站',
 '区间距离（米）',
 '方向',
 '安河桥北――北宫门',
 '北宫门――西苑',
 '西苑――圆明园',
 '圆明园――北京大学东门',
 '北京大学东门――中关村',
 '中关村――海淀黄庄',
 '海淀黄庄――人民大学',
 '人民大学――魏公村',
 '魏公村――国家图书馆',
 '国家图书馆――动物园',
 '动物园――西直门',
 '西直门――新街口',
 '新街口――平安里',
 '平安里――西四',
 '西四――灵境胡同',
 '灵境胡同――西单',
 '西单――宣武门',
 '宣武门――菜市口',
 '菜市口――陶然亭',
 '陶然亭――北京南站',
 '北京南站――马家堡',
 '马家堡――角门西',
 '角门西――公益西桥']

In [86]:
import pandas as pd 
departure=[station[0] for station in stationlist_new]
destination=[station[1] for station in stationlist_new]
distancelist=[distance.get_text() for distance in soup.find(id='sub3').select("div  td") if distance.get_text().isnumeric() ]
station_distance_df=pd.DataFrame({'departure': departure,
                                'destination':destination,
                                'distance':distancelist})


In [87]:
station_distance_df.head()

,departure,destination,distance
0,天通苑北,天通苑,939
1,天通苑,天通苑南,965
2,天通苑南,立水桥,1544
3,立水桥,立水桥南,1305
4,立水桥南,北苑路北,1286


- The if statement ***if len(stationlist)!=len(distancelist):*** takes care of the one subway line whose raw data format is off.   
- use ***.split('-')*** so we split the line  

- Also, the distance list only provided distances for one direction. So we create another dataframe to add the distances for the other direction. 

In [239]:
#this works 
distances_data_dict={}
distances_data_df=pd.DataFrame()
for oneid in id_list:
    stationlist=[station.get_text() for station in soup.find(id=oneid).select("div th")]
    #print(stationlist)
    #print(len(stationlist))
    distancelist=[distance.get_text() for distance in soup.find(id=oneid).select("div  td") if distance.get_text().isnumeric() ]
    #print(distancelist)
    if len(stationlist)!=len(distancelist):
        print('oops we have a problem')
        print(oneid)
        print(stationlist)
        stationlist=stationlist[3:]
        print(stationlist)
    stationlist_new=[station.split('――') for station in stationlist]
    departure=[station[0] for station in stationlist_new]
    #print(departure)
    destination=[station[1] for station in stationlist_new]
    #print(destination)
    distancelist=[distance.get_text() for distance in soup.find(id=oneid).select("div  td") if distance.get_text().isnumeric() ]
    station_distance_df=pd.DataFrame({'departure': departure,
                                'destination':destination,
                                'distance':distancelist})
    station_distance_df_2=pd.DataFrame({'departure': destination,
                                'destination':departure,
                                'distance':distancelist})
    #print(station_distance_df)
    #distances_data_dict[oneid]=station_distance_df
    distances_data_df=pd.concat([distances_data_df,station_distance_df,station_distance_df_2])

oops we have a problem
sub2
['起始/终到车站', '区间距离（米）', '方向', '安河桥北――北宫门', '北宫门――西苑', '西苑――圆明园', '圆明园――北京大学东门', '北京大学东门――中关村', '中关村――海淀黄庄', '海淀黄庄――人民大学', '人民大学――魏公村', '魏公村――国家图书馆', '国家图书馆――动物园', '动物园――西直门', '西直门――新街口', '新街口――平安里', '平安里――西四', '西四――灵境胡同', '灵境胡同――西单', '西单――宣武门', '宣武门――菜市口', '菜市口――陶然亭', '陶然亭――北京南站', '北京南站――马家堡', '马家堡――角门西', '角门西――公益西桥']
['安河桥北――北宫门', '北宫门――西苑', '西苑――圆明园', '圆明园――北京大学东门', '北京大学东门――中关村', '中关村――海淀黄庄', '海淀黄庄――人民大学', '人民大学――魏公村', '魏公村――国家图书馆', '国家图书馆――动物园', '动物园――西直门', '西直门――新街口', '新街口――平安里', '平安里――西四', '西四――灵境胡同', '灵境胡同――西单', '西单――宣武门', '宣武门――菜市口', '菜市口――陶然亭', '陶然亭――北京南站', '北京南站――马家堡', '马家堡――角门西', '角门西――公益西桥']


## create a sheet that aggregate all the depature and destination data 

In [242]:
distances_data_df.head(40)

,departure,destination,distance
0,苹果园,古城,2606
1,古城,八角游乐园,1921
2,八角游乐园,八宝山,1953
3,八宝山,玉泉路,1479
4,玉泉路,五棵松,1810
5,五棵松,万寿路,1778
6,万寿路,公主坟,1313
7,公主坟,军事博物馆,1172
8,军事博物馆,木樨地,1166
9,木樨地,南礼士路,1291


In [243]:
distances_data_df.columns

Index(['departure', 'destination', 'distance'], dtype='object')

In [244]:
distances_data_df.describe()

,departure,destination,distance
count,660,660,660
unique,288,288,290
top,三元桥,三元桥,1100
freq,5,5,8


## put station info into dictionary 
We use dictionary, so there won't be repetition. We find the neighbor stations for every station. Neighbor station is indirectly defined from the distance data -- if there is no distances between two stations, they are not immediately linked by any subway lines.   
We see that there are 288 city_connection keys. That is consistent with the distance_data_df dataframe which indicates 288 unique departure or destination station. 

In [245]:
from collections import defaultdict
city_connection = defaultdict(list)

In [246]:
city_connection

defaultdict(list, {})

In [247]:
for i,row in distances_data_df.iterrows():                
    city_connection[row['destination']].append(row['departure'])
    city_connection[row['departure']].append(row['destination'])
    

In [257]:
len(city_connection)

288

In [258]:
len(city_connection['三元桥'])

10

In [248]:
city_connection

defaultdict(list,
            {'古城': ['苹果园', '八角游乐园', '苹果园', '八角游乐园'],
             '苹果园': ['古城', '古城'],
             '八角游乐园': ['古城', '八宝山', '古城', '八宝山'],
             '八宝山': ['八角游乐园', '玉泉路', '八角游乐园', '玉泉路'],
             '玉泉路': ['八宝山', '五棵松', '八宝山', '五棵松'],
             '五棵松': ['玉泉路', '万寿路', '玉泉路', '万寿路'],
             '万寿路': ['五棵松', '公主坟', '五棵松', '公主坟'],
             '公主坟': ['万寿路',
              '军事博物馆',
              '万寿路',
              '军事博物馆',
              '莲花桥',
              '西钓鱼台',
              '莲花桥',
              '西钓鱼台'],
             '军事博物馆': ['公主坟',
              '木樨地',
              '公主坟',
              '木樨地',
              '白堆子',
              '北京西站',
              '白堆子',
              '北京西站'],
             '木樨地': ['军事博物馆', '南礼士路', '军事博物馆', '南礼士路'],
             '南礼士路': ['木樨地', '复兴门', '木樨地', '复兴门'],
             '复兴门': ['南礼士路', '西单', '南礼士路', '西单', '阜成门', '长椿街', '阜成门', '长椿街'],
             '西单': ['复兴门',
              '天安门西',
              '复兴门',
              '天安门西',
  

### bfs algorithm 

In [130]:
def bfs(graph, start):
    """
    breath first search
    """
    visited = [start]
    
    seen = set()
    
    while visited:
        frontier = visited.pop() #
        
        if frontier in seen: continue
        
        for successor in graph[frontier]:
            if successor in seen: continue
            #print(successor)
            
            #visited = visited + [successor] # 我们每次扩展都扩展最新发现的点 -> depth first
            visited = [successor] + visited # 我们每次扩展都先考虑已经发现的 老的点 -> breath first
            
            # 所以说，这个扩展顺序其实是决定了我们的深度优先还是广度优先
    
        seen.add(frontier)
    
    return seen

In [259]:
type(bfs(city_connection,'欢乐谷景区'))
# it found all the stations that can be connected to '欢乐谷景区' so it's quite long. 

set

Make sense also to see that this station can be linked to all 288 stations...because we know that this is no subgraph is the subway graph here. 

In [261]:
len(bfs(city_connection,'欢乐谷景区'))

288

## add a sorting algorithm to pick the shortest 

### note: not quite understanding how this sorting works 

In [151]:
def least_stops_first(paths):
    
    if len(paths) <= 1: return paths
    
    def get_path_distnace(path):
        distance = 0
        for station in path[:-1]:
            #distance += get_geo_distance(station, path[-1])
            distance +=1
            #use this to 
        return distance

    return sorted(paths, key=get_path_distnace)

In [253]:
def min_distances_first(paths):
    
    if len(paths) <= 1: return paths
    
    def get_path_distnace(path):
        distance = 0
        for station in path[:-1]:
            distance += get_between_stop_distance(station, path[-1])
            #distance +=1
            #use this to 
        return distance

    return sorted(paths, key=get_path_distnace)

In [154]:
distances_data_df.head()

,departure,destination,distance
0,苹果园,古城,2606
1,古城,八角游乐园,1921
2,八角游乐园,八宝山,1953
3,八宝山,玉泉路,1479
4,玉泉路,五棵松,1810


In [168]:
distances_data_df.head(10)

,departure,destination,distance
0,苹果园,古城,2606
1,古城,八角游乐园,1921
2,八角游乐园,八宝山,1953
3,八宝山,玉泉路,1479
4,玉泉路,五棵松,1810
5,五棵松,万寿路,1778
6,万寿路,公主坟,1313
7,公主坟,军事博物馆,1172
8,军事博物馆,木樨地,1166
9,木樨地,南礼士路,1291


In [190]:
#万寿路	公主坟 
df=distances_data_df
type(df[(df.departure=='万寿路')&(df.destination=='公主坟')].distance.tolist()[0])

str

In [234]:
def get_between_stop_distance(origin, destination):
    #global distances_data_df
    df=distances_data_df
    if origin==destination:
        return 0 
    '''
    elif any(df[(df.departure==origin)&(df.destination==destination)]):
        temp=df[(df.departure==origin)&(df.destination==destination)]
        if any(temp):
            dis=int(temp.distance.tolist()[0])
            return dis
    elif any(df[(df.departure==destination)&(df.destination==origin)]):
        temp=df[(df.departure==destination)&(df.destination==origin)]
        if any(temp):
            dis=int(temp.distance.tolist()[0])
            return dis
    '''
    try:
        temp=df[(df.departure==origin)&(df.destination==destination)]
        if any(temp):
            dis=int(temp.distance.tolist()[0])
            return dis
    except: 
         
        return 1000000000
    

In [251]:
get_between_stop_distance('木樨地','八角游乐园')

1000000000

### and a search algo that uses this sorting algorithm 

In [252]:
def search(start, destination, connection_grpah, sort_candidate):
    paths = [[start]]
    
    visitied = set()
    
    while paths: # if we find existing pathes
        path = paths.pop(0)
        frontier = path[-1]
        
        if frontier in visitied: continue
            
        successors = connection_grpah[frontier]
        
        for city in successors:
            if city in path: continue  # eliminate loop
                
            new_path = path + [city]
            
            paths.append(new_path)
            
            if city == destination: return new_path
        visitied.add(frontier)
        paths = sort_candidate(paths) # 我们可以加一个排序函数 对我们的搜索策略进行控制

In [152]:
search('欢乐谷景区', '奥林匹克公园', city_connection, sort_candidate=least_stops_first)

['欢乐谷景区',
 '南楼梓庄',
 '化工',
 '百子湾',
 '大郊亭',
 '九龙山',
 '大望路',
 '国贸',
 '永安里',
 '建国门',
 '朝阳门',
 '东四',
 '南锣鼓巷',
 '什刹海',
 '鼓楼大街',
 '安德里北街',
 '安华桥',
 '北土城',
 '奥体中心',
 '奥林匹克公园']

In [153]:
search('欢乐谷景区', '朝阳公园', city_connection, sort_candidate=least_stops_first)

['欢乐谷景区', '南楼梓庄', '化工', '百子湾', '大郊亭', '九龙山', '大望路', '红庙', '金台路', '朝阳公园']

In [255]:
search('欢乐谷景区', '朝阳公园', city_connection, sort_candidate=min_distances_first)

['欢乐谷景区', '南楼梓庄', '化工', '百子湾', '大郊亭', '九龙山', '大望路', '红庙', '金台路', '朝阳公园']

In [256]:
search('欢乐谷景区', '奥林匹克公园', city_connection, sort_candidate=least_stops_first)

['欢乐谷景区',
 '南楼梓庄',
 '化工',
 '百子湾',
 '大郊亭',
 '九龙山',
 '大望路',
 '国贸',
 '永安里',
 '建国门',
 '朝阳门',
 '东四',
 '南锣鼓巷',
 '什刹海',
 '鼓楼大街',
 '安德里北街',
 '安华桥',
 '北土城',
 '奥体中心',
 '奥林匹克公园']

In [254]:
search('欢乐谷景区', '奥林匹克公园', city_connection, sort_candidate=min_distances_first)

['欢乐谷景区',
 '南楼梓庄',
 '化工',
 '百子湾',
 '大郊亭',
 '九龙山',
 '大望路',
 '国贸',
 '永安里',
 '建国门',
 '朝阳门',
 '东四',
 '南锣鼓巷',
 '什刹海',
 '鼓楼大街',
 '安德里北街',
 '安华桥',
 '北土城',
 '奥体中心',
 '奥林匹克公园']